In [8]:
import os
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
import boto3
from requests_aws4auth import AWS4Auth
from openai import OpenAI
from config import *

from email import message_from_string
from email import policy
from email.utils import make_msgid
from src.services.data_extraction import fast_stream_first_n


Access over IAM (Switched Off)

In [9]:
# # Creating a session for a specific local user "rag-admin-user"
# session = boto3.Session(profile_name="rag-admin-user")

# # Accessing STS to assume the role with the right permissions"
# sts = session.client("sts")
# response = sts.assume_role(
#     RoleArn="arn:aws:iam::169189304433:role/RedcoatExpressRagAdmin",
#     RoleSessionName="OpenSearchUpload"
# )

# # Extracting credentials from the session
# credentials = response["Credentials"]

# # Passing credentials to AWSAUTH so we can authenticate to OpenSearch
# awsauth = AWS4Auth(
#     credentials["AccessKeyId"],
#     credentials["SecretAccessKey"],
#     REGION,
#     "es",
#     session_token=credentials["SessionToken"]
# )

# # Creating OpenSearch client with A.) my created OpenSearch domain and B.) my AWSAUTH credentials
# client = OpenSearch(
#     hosts=[{"host": DOMAIN, "port": 443}],
#     http_auth=awsauth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )

# # Testing connection:
# print(client.cat.indices(format="json"))

In [10]:
# // {
# //     "Version": "2012-10-17",
# //     "Statement": [
# //         {
# //             "Effect": "Allow",
# //             "Principal": {
# //                 "AWS": "arn:aws:iam::141472272439:role/Cradle_S3_Access_Role"
# //             },
# //             "Action": [
# //                 "s3:GetObjectAcl",
# //                 "s3:PutObject",
# //                 "s3:DeleteObject"
# //             ],
# //             "Resource": "arn:aws:s3:::uk-eoa-pipeline/*"
# //         },
# //         {
# //             "Effect": "Allow",
# //             "Principal": {
# //                 "AWS": "arn:aws:iam::141472272439:role/Cradle_S3_Access_Role"
# //             },
# //             "Action": "s3:ListBucket",
# //             "Resource": "arn:aws:s3:::uk-eoa-pipeline"
# //         }
# //     ]
# // }

In [84]:
generator =fast_stream_first_n(mbox_path, num_emails)

In [146]:

print(next(generator))

# X-Gmail-Labels: Bin,Category updates,Unread
# X-Gmail-Labels: Archived,Important,Opened,Category personal,wfs
# X-Gmail-Labels: Spam,Category promotions,Unread
# 


From 1831546495915513650@xxx Thu May 08 10:07:32 +0000 2025
X-GM-THRID: 1831546495915513650
X-Gmail-Labels: Spam,Category personal,Unread
Delivered-To: simon.leadbeatter@redcoat.org.uk
Received: by 2002:a05:622a:5b0f:b0:479:3700:fb53 with SMTP id ea15csp375010qtb;
        Thu, 8 May 2025 03:07:57 -0700 (PDT)
X-Forwarded-Encrypted: i=2; AJvYcCVfhwGLOqYDwr1Km7d4ZSDHGxYIlf+F7d+KyZMIuAli8iIsEo/e7YVI4Y0arzR14bhFTjqjmtjp4kcYxG0oGKjZiw==@redcoat.org.uk
X-Google-Smtp-Source: AGHT+IEwbULigajnsbzFWUEDgCONN6JPDJDyEXeHdXbuu27hq6zmx2ufsMCuS6vFzX6TA+0Q++Cz
X-Received: by 2002:a17:90b:1c04:b0:309:d115:b5f7 with SMTP id 98e67ed59e1d1-30aac226f3bmr10697411a91.24.1746698852711;
        Thu, 08 May 2025 03:07:32 -0700 (PDT)
ARC-Seal: i=1; a=rsa-sha256; t=1746698852; cv=none;
        d=google.com; s=arc-20240605;
        b=lD8lQiS6v35iYwp3X1QqiJBnvPToV4lM+MDC9XzDjavXkQCCUILwNmS2GiljmbMqII
         hViKgHwHbgCnVn4GEchV0T1KOC8N6dZXvtkQ80VPp2uofgJCGAE6WKYvgDwg3Tb+P9n3
         FXSNzVbgGc0lt6Ymt73Zz41s+cVnL8g

In [13]:
def parse_message_to_dict(raw_str, attachments_dir, n_char=None):
    # Parse into an EmailMessage using the default policy
    email_message = email.message_from_string(raw_str, policy=policy.default)

    # CHANGED: debug—print every header so you can spot the folder/label field
    for name, value in email_message.items():
        print(f"{name}: {value}")

